# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset


from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import shutil

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


In [4]:
ws = Workspace.from_config()

experiment_name = 'hyperdrive-run'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.core import Environment

ml_env = Environment.from_conda_specification(name = 'ml-env', file_path = './dependencies.yml')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
# Specify parameter sampler
ps = RandomParameterSampling( {  
    "C": choice(0.001, 0.01, 0.1, 1, 10, 20, 50, 100),
    "max_iter": choice(50, 75, 100,), 
    "solver" : choice('liblinear', 'sag', 'lbfgs', 'saga')
    }
)

# Specify a Policy: check the job every 2 iterations and if the primary metric falls outside the top 50% range, 
# Azure ML should terminate the job
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)


est = ScriptRunConfig(
    source_directory='.', 
    compute_target=compute_target, 
    script='train.py',
    environment=ml_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=est, 
    hyperparameter_sampling=ps,
    primary_metric_name="AUC_weighted", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    max_total_runs=10, 
    max_concurrent_runs=5)

In [8]:
hypdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hypdrive_run).show()
hypdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_14476565-ccef-423f-87bd-35f019d3782d
Web View: https://ml.azure.com/runs/HD_14476565-ccef-423f-87bd-35f019d3782d?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-144814/workspaces/quick-starts-ws-144814&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-14T17:34:57.067170][API][INFO]Experiment created<END>\n""<START>[2021-05-14T17:34:57.491260][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-05-14T17:34:57.666240][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_14476565-ccef-423f-87bd-35f019d3782d
Web View: https://ml.azure.com/runs/HD_14476565-ccef-423f-87bd-35f019d3782d?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-144814/workspaces/quick-starts-ws-144814&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "error

{'runId': 'HD_14476565-ccef-423f-87bd-35f019d3782d',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-14T17:34:56.902068Z',
 'endTimeUtc': '2021-05-14T17:47:02.794567Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:

import joblib
# Get your best run and save the model from that run.

hd_best_run = hypdrive_run.get_best_run_by_primary_metric()
hd_best_model = hd_best_run.register_model(model_name="hd_best_model" , model_path="./")
parameter_values = hd_best_run.get_details()

print("Best Run Id: ", hd_best_run.id)
print("Best run details ", parameter_values)
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

best_run_metrics = hd_best_run.get_metrics() #will prodocue an error, see https://github.com/Azure/MachineLearningNotebooks/issues/737 

print("Best run metrics: ", best_run_metrics)
print("Accuracy: ", best_run_metrics["Accuracy"])


Best Run Id:  HD_14476565-ccef-423f-87bd-35f019d3782d_8
Best run details  {'runId': 'HD_14476565-ccef-423f-87bd-35f019d3782d_8', 'target': 'capstone-cluster', 'status': 'Completed', 'startTimeUtc': '2021-05-14T17:41:10.810417Z', 'endTimeUtc': '2021-05-14T17:42:21.496205Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '9bbf3a30-cadd-44ef-a34d-bad07b9075c2', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '10', '--max_iter', '100', '--solver', 'liblinear'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'capstone-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'priority': None, 'credentialPassthrough': False, 'identity': None,

Exception: Malformed metric value

## Model Deployment

Since the hyperparameter optimization did not result in a better, I decided to deploy the AutoML model as a web service. 
The best AUC_weighted metric for a HD model was: 0.8273177864507736